In [9]:
# !py -3 -m pip install --upgrade Jinja2==3.0.3

In [10]:
from datetime import datetime, timedelta
import time
import os
import json
from jinja2 import Environment, FileSystemLoader
from google.cloud import bigquery
import logging
import re

In [11]:
def read_contents_file(file_path, encoding_scheme='cp1252'):
    inputfile = open(file_path, 'r', encoding=encoding_scheme)
    return inputfile.read()

In [12]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = file_data
        file.write(file_string)

In [13]:
with open('config/lob_config.json') as json_lob_config:
    config = json.load(json_lob_config)

lob = config['lob']
lob_lower = lob.lower().strip()
domain_abbr = config['domain_abbr']
domain_syn_names = config['domain_syn_names']
project_id = config['project_id']
monitoring_dataset = config['monitoring_dataset']
authviews_dataset = config['authviews_dataset']
audit_dataset = config['audit_dataset']

yaml_template_path = config['yaml_template_path']
jinja_extension = config['jinja_extension']
sql_extension = config['sql_extension']
output_path_folder = config['output_path_folder']
parameterized_folder = config['parameterized_folder']
qualified_folder = config['qualified_folder']

domain_abbr_lower = domain_abbr.lower().strip()
domain_case_expr = [f"            WHEN '{domain_synonym}' THEN '{domain_abbr}'" for domain_synonym in domain_syn_names]
regexp_number_prefix = re.compile(r'[0-9]*_(.*)',re.IGNORECASE)

replace_params = {
    'params' : {
        "param_monitoring_views_dataset_name": f"{project_id}.{monitoring_dataset}",
        "param_auth_base_views_dataset_name": f"{project_id}.{authviews_dataset}",
        f"param_{lob_lower}_audit_dataset_name": f"{project_id}.{audit_dataset}"
    }
}

lob_dict = {
    "monitoring_dataset_param"  :   "{{ params.param_monitoring_views_dataset_name }}",
    "authviews_dataset_param"   :   "{{ params.param_auth_base_views_dataset_name }}"
}

lob_dict.update(
        {
            "lob_name":f"{lob}",
            "lob_name_lower":f"{lob_lower}",
            "lob_syn_regex":'|'.join(domain_syn_names),
            "domain_name":f"{domain_abbr}",
            "domain_abbr_lower":f"{domain_abbr_lower}",
            "domain_case_expr":'\n'.join(domain_case_expr),
            "audit_dataset_param":"{{ params.param_"+lob_lower+"_audit_dataset_name }}"
        }
    )

In [14]:
def execute_sql_files(sql_file_path):

    client = bigquery.Client(project=project_id)
    with open(sql_file_path, 'r') as sql_file:
        file_name = os.path.basename(sql_file_path)
        sql_query = sql_file.read()
        try:         
            client.query(sql_query, project=project_id, location='US').result()
            logging.info(f"SQL file {file_name} executed successfully.")
            print(f"SQL file {file_name} executed successfully.")
        except Exception as e:
            logging.error(f"Error executing SQL file {file_name}: {e}")
            print(f"Error executing SQL file {file_name}: {e}")

In [15]:
#get the DDLs for the tables from teradata 
def gen_bq_views_ddl():
    dt1 = datetime.now()

    try:
        parameterized_path = f"{output_path_folder}\{lob.strip()}\{parameterized_folder}"
        if not os.path.exists(parameterized_path):
            os.makedirs(parameterized_path)
        qualified_path = f"{output_path_folder}\{lob.strip()}\{qualified_folder}"
        if not os.path.exists(qualified_path):
            os.makedirs(qualified_path)

        # Jinja2 Environment
        env = Environment(loader = FileSystemLoader(yaml_template_path),   trim_blocks=True, lstrip_blocks=True)  

        for filename in os.listdir(yaml_template_path):
            template = env.get_template(filename)
            sql_filename = filename.replace(jinja_extension,sql_extension)
            parameterized_filename_path = f"{parameterized_path}\{sql_filename}"
            write_file_local(parameterized_filename_path, template.render(lob_dict))

        print("Project ID : {}".format(project_id))
        
        # Jinja2 Environment
        env = Environment(loader = FileSystemLoader(parameterized_path),   trim_blocks=True, lstrip_blocks=True)  

        for sql_filename in os.listdir(parameterized_path):
            template = env.get_template(sql_filename)
            regexp_number_prefix_match = regexp_number_prefix.match(sql_filename)
            if regexp_number_prefix_match is not None:
                # Rename the Templated File
                parameterized_filename_path = f"{parameterized_path}\{sql_filename}"
                sql_filename = ''.join(regexp_number_prefix_match.groups())
                parameterized_filename_newpath = f"{parameterized_path}\{sql_filename}"
                if os.path.exists(parameterized_filename_newpath):
                    os.remove(parameterized_filename_newpath)
                os.rename(parameterized_filename_path, parameterized_filename_newpath)
            qualified_filename_path = f"{qualified_path}\{sql_filename}"
            write_file_local(qualified_filename_path, template.render(replace_params))
            # execute_sql_files(qualified_filename_path)

    except Exception as e1:
        print(e1)
        print("LOB : {}".format(lob))
        print("File Name : {}".format(filename))
        pass
    
    dt2 = datetime.now()
    print(dt2-dt1)

In [16]:
print("Begin of Processing")

gen_bq_views_ddl()

print("End of Processing")

Begin of Processing
Project ID : hca-hin-dev-cur-clinical
0:00:00.451035
End of Processing
